In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *
context = Context('rf-revisit', gpu_start=6, dep_var=dep_var1, debug=0, mulr=1, fn_feather='tbmData/feather/removed_redundent1-full-30-no_noise.feather')
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

In [ ]:
cyc_cont,train_idx_tile,valid_idx_tile,dep_var = context.cyc_cont, context.train_idx_tile, context.valid_idx_tile, context.dep_var
train, val = cyc_cont.iloc[train_idx_tile], cyc_cont.iloc[valid_idx_tile]
X_train, y_train = train.drop(columns=dep_var), train.loc[:,dep_var]
X_valid, y_valid = val.drop(columns=dep_var), val.loc[:,dep_var]

In [ ]:
%%cython
def my_polyx(double a, double b):
    return 10.5 * a + 3 * (b**2)

In [ ]:
def my_poly(a,b):
    return 10.5 * a + 3 * (b**2)

In [ ]:
%load_ext cython

In [ ]:
%%cython
import numpy as np
cimport numpy as np

def million_floats(np.ndarray v):
    for i in range(v.shape[0]): 
        v[i] += 2.0

In [ ]:
v = np.zeros(int(1e8))

In [ ]:
million_floats(v)

In [ ]:
im = np.array([[1,2,3],[4,5,6],[7,8,9]]); im

In [ ]:
rr, cc = (0,1,2), (0,1,2)
im[rr[1:], cc[1:]] - im[rr[:-1], cc[:-1]]

In [ ]:
my_polyx(2, 3)

In [ ]:
%timeit my_poly(10, 2)
%timeit my_polyx(10, 2)

In [ ]:
from sklearn.ensemble import forest
from sklearn.ensemble import RandomForestRegressor
def set_rf_samples(n):
    """ Changes Scikit learn's random forests to give each tree a random sample of
    n random rows.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n))

def reset_rf_samples():
    """ Undoes the changes produced by set_rf_samples.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n_samples))
def print_score(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)
    
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)
mapd = MAPD(context.stat_y)

In [ ]:
# set_rf_samples(100)
reset_rf_samples()

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=3)
%time m.fit(X_train, y_train)
print(m.score(X_train, y_train), m.score(X_valid, y_valid), m.oob_score_)

In [ ]:
p_train, p_valid = m.predict(X_train), m.predict(X_valid)
print('train:',to_np(mapd(tensor(p_train).float(), tensor(y_train.values).float())))
print('valid: ', to_np(mapd(tensor(p_valid).float(), tensor(y_valid.values).float())))
display_all(rf_feat_importance(m, X_valid))

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=5)
%time m.fit(X_train, y_train)
print(m.score(X_train, y_train), m.score(X_valid, y_valid), m.oob_score_)

In [ ]:
p_train, p_valid = m.predict(X_train), m.predict(X_valid)
print(mapd(tensor(p_train).float(), tensor(y_train.values).float()), mapd(tensor(p_valid).float(), tensor(y_valid.values).float()))
display_all(rf_feat_importance(m, X_valid))

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=1)
m.fit(X_train, y_train)
print(m.score(X_train, y_train), m.score(X_valid, y_valid), m.oob_score_)

In [ ]:
p_train, p_valid = m.predict(X_train), m.predict(X_valid)
print(mapd(tensor(p_train).float(), tensor(y_train.values).float()), mapd(tensor(p_valid).float(), tensor(y_valid.values).float()))
display_all(rf_feat_importance(m, X_valid))